# Konvolutivne mreže, klasifikacija slika

Materijali su zasnovani na zvaničnom TensorFlow [tutorial](https://www.tensorflow.org/tutorials/images/classification)-u.

U ovom primeru ćemo napraviti  klasifikator koji će da vrši klasifikaciju
između mačaka i pasa. Biće prikazano kako možemo da veštački povećamo skup
dostupnih podataka korišćenjem `tf.keras.preprocessing.image.ImageDataGenerator`
funkcionalnosti koju nudi keras. Prikazaćemo i regularizaciju izostavljanjem
(eng. dropout), kao i kako raditi sa slikama koje se nalaze na disku.


## Učitavanje paketa

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

## Učitavanje podataka

Koristićemo <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs Cats</a> skup podataka sa platforme Kaggle.

Umesto korišćenja `tf.keras.utils.get_file` možete i ručno preuzeti datoteu,
smestiti je negde na disk, i ažurirati putanje da pokazuju na nju.


In [0]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

Skup podataka ima sledeću strukturu:

<pre>
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

In [4]:
PATH

'/root/.keras/datasets/cats_and_dogs_filtered'

Definišemo neophodne putanje do skupova za obučavanje, validaciju i klasa unutar njih.

In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [0]:
train_cats_dir = os.path.join(train_dir, 'cats') 
train_cats_dir = os.path.join(train_dir, 'cats') 
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

### Razumevanje podataka

Pogledajmo osnovne informacije o skupu podataka kao što su broj instanci,
broj instanci po klasi, broj klasa, kako izgledaju slike, rezolucija slika i slično.

In [0]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [0]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

Treba izbegavati korišćenje konstnaci u kodu, pa možemo definisati
promenljive koje ćemo na dalje koristiti.

In [0]:
batch_size = 128
epochs = 20
IMG_HEIGHT = 150
IMG_WIDTH = 150

## Priprema podataka

Kako bi slike prosledili mreži, neophodno je da ih
učitamo i transformišemo u tensore.

Kako bi olakšali ovaj proces, možemo koristiti `ImageDataGenerator` klasu
koju nudi `tf.keras`. Ona nudi funkcionalnost čitanja slika sa diska
i primene definisanih koraka pretprocesiranja čiji rezultat je tensor
koji se može proslediti mreži.

Naredni deo konstruiše `ImageDataGenerator` generatore za naš skup podataka.
Definišemo `rescale=1./255` čije će generator da pomnoži svaki piksel slike
sa vrednošću $\frac{1}{255}$.

In [0]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

Funkcija `flow_from_directory` učitava slike sa diska, primenjuje skaliranje i vrši skaliranje slika u potrebnu veličinu (koju definiše korisnik).
Rezultat rada funkcije je iterator koji možemo da koristiti da dobijamo
`batch_size` slika nad svakim pozivom iteratora.

In [0]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [0]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [13]:
train_data_gen

### Vizuelizacija slika

Prikažimo nekoliko slika.

In [0]:
sample_training_images, sample_training_labels = next(train_data_gen)

In [15]:
np.unique(sample_training_labels)

array([0., 1.], dtype=float32)

Funkcija `next` (python standardna biblioteka) nam vraća podskup (batch)
iz skupa podataka. Povratna vrednost je u obliku `(x_train, y_train)`.

In [0]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [0]:
plotImages(sample_training_images[:5])

## Definisanje modela

Slično kao u prethodnom primeru, napravićemo nekoliko slojeva
koji se sastoji od parova konvolucija-agregacija.
Nakon toga ćemo dodati `Flatten` sloj da sve prebacimo u vektor koji će dalje
biti prosleđen binarnom klasifikatoru.

U principu umesto 1 neurona na kraju možemo koristiti i dva (kao u ranijim
primerima sa više klasa), ali nema potrebe za tim. U praksi se često koristi
jedan neuron i posmatra se njegova aktivacija. Postavićemo mu sigmoidnu
aktivacionu funkciju jer podseća na funkciju verovatnoće.

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

### Compile the model

For this tutorial, choose the *ADAM* optimizer and *binary cross entropy* loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument.

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

### Pregled modela

In [0]:
model.summary()

### Obučavanje modela

Pošto naši podaci nisu učitani u radnu memoriju kao u prošlom primeru već
koristimo generator koji ih po potrebi učitava, koristićemo funkciju
`fit_generator` koja očekuje da joj se prosledi generator
pomoću kojeg će umeti da dođe do neophodnih podataka prilikom
obučavanja mreže.

In [21]:
print(f"total_train: {total_train}")
print(f"total_val: {total_val}")
print(f"batch_size: {batch_size}")
print(f"steps_per_epoch: {total_train // batch_size}")

total_train: 2000
total_val: 1000
batch_size: 128
steps_per_epoch: 15


In [0]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Pregled obučavanja mreže

Now visualize the results after training the network.

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Možemo primetiti da je greška na validacionom skupu počela da raste nakon nekih
10ak epoha, kao i da tačnost počinje drastično da odstupa od skupa za obučavanje.
Ovo je jasan znak da dolazi do preprilagođavanja. Ovo je i očekivano sa
obzirom na kompleknost modela koji koristimo i broj instanci u skupu
podataka koji nam je dostupan.

<a href="https://imgflip.com/i/40mxrp"><img src="https://i.imgflip.com/40mxrp.jpg" title="made at imgflip.com"/></a>

## Augmentacija podataka

Problem malog broja podataka možemo ublažiti tako što možemo generisati
još podataka na osnovu postojećih korišćenjem nekih definisanih
transformacija.

`ImageDataGenerator` pruža podršku za ovo uz dosta jednostavan interfejs
za korišćenje.

### Augmentacija i vizuelizacija podataka

Pre nego što definišemo niz transformacija i pustimo obučavanje modela,
pogledajmo malo detaljnije kako funkcioniše `ImageDataGenerator`.

Na primer, dodajmo horizontalno obrtanje slike.

### Horizontal flip

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))


Uzećemo jednu sliku iz skupa i ponoviti je 5 puta kako bi primetili
kako se održava transformacija koju smo definisali.

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Rotacija slika

Možemo dodati nasumičnu rotaciju slike u intervalu [-45, 45].

In [0]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Zumiranje slike

Može se primeniti i transformacija koja dodaje zumiranje nad slikom.

In [0]:
# zoom_range from 0 - 1 where 1 = 100%.
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5) # 

In [0]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

In [0]:
plotImages(augmented_images)

### Definisanje nekoliko transformacija

Sada ćemo primeniti nekoliko transformacija i pripremiti generator za
slike.

In [0]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )

In [0]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

In [0]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Kreiranje validacionog generatora

U suštini, i ne želimo da dodajemo neke transformacije nad validacionim
slikama, tako da će nam generator za validacioni slike praktično
ostati kao u prošlom primeru.

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [0]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary')

## Dropout

Regularizacija izostavljanjem (eng. dropout) je tehnika regularizacije
kojom se može poboljšati stabilnost optimizacije odnosno obučavanje
neuronske mreže. Dropout se definiše nad slojem tako što se zada
parametar između 0 i 1. Ako postavimo da je dropout 0.1, tokom
treninga će u tom sloju nasumično biti odabrano 10% jedinica kojima
će biti ubijem izlaz. Ovime povećavamo robusnost modela jer ga teramo
da se tokom treninga više adaptira na ove promene koje dropout uvodi.

Više informacija o dropout-u možete pronaći [ovde](https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/).

## Definisanje nove mreže koja ima Dropout

Postavićemo dropout nakon parova konvolucije i agregacije sa vrednošću 0.2.

In [0]:
model_new = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

### Kompilacija modela

In [0]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

model_new.summary()

### Obučavanje modela

Nakon izmena koje smo uveli, konačno je vreme da ponovo obučimo model!

<img src="https://i.stack.imgur.com/p6uNz.jpg">

In [0]:
history = model_new.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

### Evaluacija modela

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Možemo primetiti da je model sada u dosta boljoj situaciji u odnosu na prošli pokušaj! Ima smisla produžiti broj epoha, koristiti `EarlyStopping` i
videti koliko je u stanju model da nauči.

DOMAĆI: Pokušajte da povećate tačnost!